In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [50]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = ""

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS, timeout=30)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
...
amazon_df = pd.DataFrame.from_dict(d)
amazon_df.fillna(value=np.nan, inplace=True)  # Replace empty strings with NaN
amazon_df = amazon_df.dropna(subset=['title'])  # Drop rows with NaN in the 'title' column
amazon_df.to_csv("amazon_data.csv", header=True, index=False)
amazon_df.to_excel("amazon_data.xlsx", index=False)


In [51]:
amazon_df

,title,price,rating,reviews,availability
0,,,,,Not Available
1,,,,,Not Available
2,,,,,Not Available
3,"OnePlus 12R, 16GB RAM+256GB, Dual-SIM, US Fact...",$529.99,4.4 out of 5 stars,56 ratings,In Stock
4,,,,,Not Available
5,,,,,Not Available
6,,,,,Not Available
7,,,,,Not Available
8,,,,,Not Available
9,,,,,Not Available


# 1 Data Visualization


In [68]:
# Đọc dữ liệu từ tệp CSV
amazon_df = pd.read_csv("amazon_data.csv")

# Thay thế các giá trị NaN trong cột 'price' bằng giá trị trung bình của cột
mean_price = amazon_df['price'].astype(float).mean()
amazon_df['price'].fillna(mean_price, inplace=True)

# Hiển thị lại DataFrame sau khi xử lý
print(amazon_df)


Empty DataFrame
Columns: [title, price, rating, reviews, availability]
Index: []


C:\Users\locn3\AppData\Local\Temp\ipykernel_1288\1983693164.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['price'].fillna(mean_price, inplace=True)


# 2 Count the number of available and unavailable products

In [69]:
availability_counts = amazon_df['availability'].value_counts()

print("Count the number of available and unavailable products:")
print(availability_counts)

Count the number of available and unavailable products:
Series([], Name: count, dtype: int64)


# 3. Determine the product with the highest and lowest price.

In [70]:
# Identify products with the highest and lowest prices
max_price_product = amazon_df.loc[amazon_df['price'].idxmax()]
min_price_product = amazon_df.loc[amazon_df['price'].idxmin()]

# Display product information with the highest and lowest price
print("Highest priced product:")
print(max_price_product)
print("\nLowest priced product:")
print(min_price_product)


ValueError: attempt to get argmax of an empty sequence

# 4. Sort products by price

In [ ]:
# Define the price classification function
def categorize_price(price):
     if price < 100:
         return 'Under $100'
     elif 100 <= price < 500:
         return 'From 100$ to 500$'
     else:
         return 'Over $500'

# Apply the classification function to column 'price' to create a new column 'price_category'
amazon_df['price_category'] = amazon_df['price'].apply(categorize_price)

# Display products sorted by price
price_category_counts = amazon_df['price_category'].value_counts()
print("Sort products by price:")
print(price_category_counts)

# 5. The chart shows the price distribution of products

In [ ]:
# Convert column 'price' to numeric form for calculation
amazon_df['price'] = amazon_df['price'].replace(r'[^\d.]', '', regex=True).astype(float)

# Create a histogram chart for the price distribution of products
plt.figure(figsize=(10, 6))
plt.hist(amazon_df['price'], bins=20, color='orange', edgecolor='black', alpha=0.7)
plt.title('Price distribution of products on Amazon')
plt.xlabel('Price')
plt.ylabel('Product quantity')
plt.show()

